In [1]:
from pyomo.environ import *
from itertools import permutations

In [2]:
m = AbstractModel()
"""
Parameters:
  z
  |_y
 /
x
x -> Depth  D
y -> Width  W
z -> Height H
"""

m.N_value = Param(domain=PositiveIntegers)
m.W_max = Param(domain=PositiveReals)
m.H_max = Param(domain=PositiveReals)
m.D_max = Param(domain=PositiveReals)
m.N = RangeSet(m.N_value)
m.N_N = Set(initialize=m.N*m.N, filter=lambda _,i,j: i<j)

m.D = Param(m.N, domain=PositiveReals)
m.W = Param(m.N, domain=PositiveReals)
m.H = Param(m.N, domain=PositiveReals)

m.D_rot = Var(m.N, domain=NonNegativeReals, bounds=(0,m.W_max*m.H_max*m.D_max), initialize=0)
m.W_rot = Var(m.N, domain=NonNegativeReals, bounds=(0,m.W_max*m.H_max*m.D_max), initialize=0)
m.H_rot = Var(m.N, domain=NonNegativeReals, bounds=(0,m.W_max*m.H_max*m.D_max), initialize=0)

m.x = Var(m.N, domain=NonNegativeReals, bounds=(0,m.D_max), initialize=0)
m.y = Var(m.N, domain=NonNegativeReals, bounds=(0,m.W_max), initialize=0)
m.z = Var(m.N, domain=NonNegativeReals, bounds=(0,m.H_max), initialize=0)

m.lt = Var(domain=PositiveReals)
m.obj = Objective(expr=m.lt)

@m.Constraint(m.N)
def depth_constr(m,i):
    return m.x[i]+m.D_rot[i] <= m.D_max

@m.Constraint(m.N)
def width_constr(m,i):
    return m.y[i]+m.W_rot[i] <= m.W_max

@m.Constraint(m.N)
def objective_constr(m,i):
    return m.z[i]+m.H_rot[i] <= m.lt

@m.Disjunction(m.N_N)
def direction_disjunct(m,i,j):
    return [
        m.x[i]+m.D_rot[i] <= m.x[j],
        m.x[j]+m.D_rot[j] <= m.x[i],
        m.y[i]+m.W_rot[i] <= m.y[j],
        m.y[j]+m.W_rot[j] <= m.y[i],
        m.z[i]+m.H_rot[i] <= m.z[j],
        m.z[j]+m.H_rot[j] <= m.z[i]
    ]


@m.Disjunction(m.N)
def rotation(m,i):
    """Disjunction with a disjunct for each way to combine (D,W,H)_i and (D_rot,W_rot,H_rot)_i"""
    #dimension_var = [m.D_rot, m.W_rot, m.H_rot]
    #dimension_params = [m.D, m.W, m.H]
    #Z = [zip(var_perm, dimension_params) for var_perm in permutations(dimension_vars)]
    #return [[var[i] == param[i] for (var,param) in z] for z in Z]
    return [[var[i] == param[i] for (var,param) in z] for z in [zip(var_perm, [m.D, m.W, m.H]) for var_perm in permutations([m.D_rot, m.W_rot, m.H_rot])]]

In [3]:
instance = m.create_instance('rand_params_3D.dat')
opt = SolverFactory('gdpopt')  # or TransformationFactory & gurobi
opt.solve(instance)

{'Problem': [{'Name': 'unknown', 'Lower bound': 2.85238778197, 'Upper bound': 2.852387741177995, 'Number of objectives': 1, 'Number of constraints': 741, 'Number of variables': 625, 'Number of binary variables': 546, 'Number of integer variables': 0, 'Number of continuous variables': 79, 'Number of nonzeros': None, 'Sense': 1, 'Number of disjunctions': 91}], 'Solver': [{'Name': 'GDPopt (19, 3, 11) - LOA', 'Status': 'ok', 'Message': None, 'User time': 155.55449965600002, 'System time': None, 'Wallclock time': 155.55449965600002, 'Termination condition': 'optimal', 'Termination message': None, 'Timing': Container(OA cut generation = 0.0010004920000028505, initialization = 0.3292706920000228, integer cut generation = 0.006407468000020344, main loop = 154.99565013999998, main_timer_start_time = 204.808238695, mip = 154.451686809, nlp = 0.5363212309999881, total = 155.55449965600002), 'Iterations': 1}]}

In [6]:
def set_axes_radius(ax, origin, radius):
    ax.set_xlim3d([origin[0] - radius, origin[0] + radius])
    ax.set_ylim3d([origin[1] - radius, origin[1] + radius])
    ax.set_zlim3d([origin[2] - radius, origin[2] + radius])

def set_axes_equal(ax):
    '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
      
    Source: https://stackoverflow.com/a/50664367/5616591
    '''

    limits = np.array([
        ax.get_xlim3d(),
        ax.get_ylim3d(),
        ax.get_zlim3d(),
    ])

    origin = np.mean(limits, axis=1)
    radius = 0.5 * np.max(np.abs(limits[:, 1] - limits[:, 0]))
    set_axes_radius(ax, origin, radius)

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random  # for colors
from itertools import product, combinations  # for 3D box
%matplotlib qt

def plotBox(ax, v1, v2, **kwargs):
    """Plot 3D box from two cornerpoins v1 and v2"""
    for a, b in combinations(list(product(*zip(v1,v2))),2):
        if len(np.nonzero(np.array(a)-np.array(b))[0]) == 1:
            ax.plot3D(*zip(a,b), **kwargs)

fig = plt.figure(figsize=(15, 12))
ax = fig.add_subplot(111, projection='3d')
ax.set_aspect("equal")
plt.title('Packing solution (with rotation)', fontsize=16)

ax.set_xlabel('x-axis (fixed)', fontsize=14)
ax.set_ylabel('y-axis (fixed)', fontsize=14)
ax.set_zlabel('z-axis (minimization direction)', fontsize=14)

(x,y,z) = (instance.x, instance.y, instance.z)
(D,W,H) = (instance.D_rot, instance.W_rot, instance.H_rot)

# Plot the 'containing' box, i.e. the width and depth limits
plotBox(ax, (0,0,0), (instance.D_max.value, instance.W_max.value, 0), color='red',linewidth=5)

# Plot all the model boxes
for i in instance.N:
    random.seed(i)
    random_color = [random.random() for i in range(3)]
    plotBox(ax, (x[i].value, y[i].value, z[i].value), 
                (x[i].value+D[i].value, y[i].value+W[i].value, z[i].value+H[i].value),
            color=random_color, linewidth=5, alpha=0.6)
    
set_axes_equal(ax)
plt.tight_layout()